In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt, gc, os
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/amex-default-prediction/sample_submission.csv
/kaggle/input/amex-default-prediction/train_data.csv
/kaggle/input/amex-default-prediction/test_data.csv
/kaggle/input/amex-default-prediction/train_labels.csv
/kaggle/input/amex-data-integer-dtypes-parquet-format/train.parquet
/kaggle/input/amex-data-integer-dtypes-parquet-format/test.parquet


In [2]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import os
import lightgbm
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import warnings
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, auc, RocCurveDisplay, accuracy_score
import shap
import itertools

In [3]:
#Becaurse the files are too large to read,so use the parquet file format.
train = pd.read_parquet("/kaggle/input/amex-data-integer-dtypes-parquet-format/train.parquet").groupby('customer_ID').tail(4)
test = pd.read_parquet("/kaggle/input/amex-data-integer-dtypes-parquet-format/test.parquet").groupby('customer_ID').tail(4)
train_labels = pd.read_csv("../input/amex-default-prediction/train_labels.csv")
train.head()

,customer_ID,S_2,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,...,D_136,D_137,D_138,D_139,D_140,D_141,D_142,D_143,D_144,D_145
9,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-12-04,0.868580,0,0.001930,1.007504,0.001772,0.100470,0.0,0.006073,...,-1,-1,-1,0,0,0.0,NaN,0,0.004753,0
10,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2018-01-11,0.909811,0,0.002829,1.004798,0.008175,0.098882,0.0,0.003238,...,-1,-1,-1,0,0,0.0,NaN,0,0.003004,0
11,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2018-02-21,0.937349,0,0.019837,1.008307,0.000607,0.102985,0.0,0.007454,...,-1,-1,-1,0,0,0.0,NaN,0,0.006346,0
12,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2018-03-13,0.934745,0,0.009382,1.007647,0.006104,0.135021,0.0,0.007174,...,-1,-1,-1,0,0,0.0,NaN,0,0.002970,0
22,00000fd6641609c6ece5454664794f0340ad84dddce9a2...,2017-12-29,0.861109,10,0.006711,0.819772,0.007966,0.139138,0.0,0.005235,...,-1,-1,-1,0,0,0.0,NaN,0,0.003133,0


In [4]:
#shapes
train.shape, test.shape, train_labels.shape

((1802318, 190), (3656852, 190), (458913, 2))

In [5]:
print('Number of row:'+str(len(train)))
print('Number of column:'+str(len(train.columns)))

Number of row:1802318
Number of column:190


In [6]:
train.describe()

,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,D_42,D_43,...,D_136,D_137,D_138,D_139,D_140,D_141,D_142,D_143,D_144,D_145
count,1.788975e+06,1.802318e+06,1.802318e+06,1.801953e+06,1.802318e+06,1.474135e+06,1.801953e+06,1.801953e+06,244122.000000,1.272657e+06,...,1.802318e+06,1.802318e+06,1.802318e+06,1.802318e+06,1.802318e+06,1.769701e+06,310496.000000,1.802318e+06,1.801179e+06,1.802318e+06
mean,6.411824e-01,5.795876e+00,1.355018e-01,5.991593e-01,9.770583e-02,2.307265e-01,7.299108e-02,1.495153e-01,0.180821,1.615884e-01,...,-9.245344e-01,-9.616677e-01,-9.509687e-01,1.541842e-01,2.306419e-02,1.617967e-01,0.392772,1.540855e-01,5.163485e-02,6.143194e-01
std,2.588934e-01,1.088725e+01,2.250991e-01,4.095068e-01,2.588383e-01,1.995605e-01,2.490946e-01,2.512022e-01,0.248118,2.247142e-01,...,4.128786e-01,1.936461e-01,2.664026e-01,4.081740e-01,1.542601e-01,3.513826e-01,0.240538,4.080903e-01,1.811209e-01,2.153483e+00
min,-4.589548e-01,0.000000e+00,-7.588799e+00,9.192280e-09,3.240394e-09,-5.088831e-01,0.000000e+00,1.028956e-08,-0.000332,5.429255e-07,...,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,0.000000e+00,-0.014539,-1.000000e+00,4.701217e-09,-1.000000e+00
25%,4.577217e-01,0.000000e+00,9.085041e-03,7.808472e-02,2.978019e-03,1.282375e-01,0.000000e+00,5.390869e-03,0.033292,4.301413e-02,...,-1.000000e+00,-1.000000e+00,-1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.197211,0.000000e+00,2.749411e-03,0.000000e+00
50%,6.841134e-01,0.000000e+00,3.435712e-02,8.136464e-01,5.954595e-03,1.652470e-01,0.000000e+00,1.029428e-02,0.113116,9.156333e-02,...,-1.000000e+00,-1.000000e+00,-1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.385020,0.000000e+00,5.499610e-03,0.000000e+00
75%,8.616738e-01,8.000000e+00,1.474703e-01,1.002090e+00,8.919717e-03,2.670320e-01,0.000000e+00,1.965373e-01,0.237627,1.925332e-01,...,-1.000000e+00,-1.000000e+00,-1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.571261,0.000000e+00,8.243219e-03,0.000000e+00
max,1.010000e+00,1.820000e+02,1.324060e+00,1.010000e+00,3.008229e+00,4.882418e+00,8.988807e+00,1.625262e+00,4.191119,1.011162e+01,...,7.000000e+00,1.000000e+00,6.000000e+00,1.000000e+00,1.000000e+00,1.339910e+00,2.229368,1.000000e+00,1.343326e+00,5.300000e+01


In [7]:
#count NaN or missing values in the DataFrame
print('/nCount total NaN at each column in a DataFrame: /n/n',train.isnull().sum())

/nCount total NaN at each column in a DataFrame: /n/n customer_ID          0
S_2                  0
P_2              13343
D_39                 0
B_1                  0
                ...   
D_141            32617
D_142          1491822
D_143                0
D_144             1139
D_145                0
Length: 190, dtype: int64


In [8]:
#Columns that have more than 50% of missing values
columns=train.columns[(train.isna().sum()/len(train))*100>50]

In [9]:
#Drop the columns which have more than 50% of missing values.
train = train.drop(columns, axis=1)
test = test.drop(columns, axis=1)

In [10]:
#fill the missing values, replace the null values with the values from the next or previous row. 
train = train.bfill(axis='rows').ffill(axis='rows')
test = test.bfill(axis='rows').ffill(axis='rows')

In [11]:
train.reset_index(inplace=True)
test.reset_index(inplace=True)

In [12]:
train =train.groupby('customer_ID').tail(1)
test = test.groupby('customer_ID').tail(1)

In [13]:
#Make the changes permanent
train.reset_index(drop=True, inplace=True)
test.reset_index(drop=True, inplace=True)

In [14]:
#shape
train.shape, train_labels.shape, test.shape

((458913, 174), (458913, 2), (924621, 174))

In [15]:
#Find the unique values which only appear once in the categorical columns.
categorical_col = ['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68']
for col in categorical_col:
    print(train[col].unique())
    print(test[col].unique())

[ 0  1  2 -1]
[ 0  1  2 -1]
[ 2  1  3  7  5  6  4 -1]
[ 2  3  6  1  7  5  4 -1]
[ 1  0 -1]
[ 0  1 -1]
[ 0 -1  1]
[ 0 -1  1]
[ 5  0  7  2  3  6  4 -1]
[ 0  4  5  3  7  6 -1  2]
[ 0  1 -1]
[ 1  0 -1]
[2 1]
[1 2]
[0 3 4 5 1 2]
[0 3 4 2 1 5]
[ 0  2  3 -1]
[ 3  0  2 -1]
[-1  1]
[-1  1]
[ 6  3  5  4  2  1 -1]
[ 6  4  5  2  1  3 -1]


In [16]:
#Visualizing Target
sns.countplot?